In [3]:
import pandas as pd

df_crime_yearly = pd.read_csv('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/ouput_2crime_yearly_2014_2024.csv', encoding='cp949')
df_time_final = pd.read_csv('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/ouput_2crime_time_distribution.csv', encoding='cp949')
df_place_final = pd.read_csv('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/ouput_2crime_place_distribution.csv', encoding='cp949')

print("준비완료")

준비완료


In [2]:
"""
서울시 5대 범죄 위험 시간대 가설 검증 및 시각화
(한글 폰트 문제 해결 버전)
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ============================================
# 한글 폰트 자동 설정 (macOS/Linux/Windows 호환)
# ============================================

def set_korean_font():
    """시스템에서 사용 가능한 한글 폰트 자동 감지 및 설정"""
    
    # 폰트 우선순위 리스트 (macOS → Linux → Windows)
    font_candidates = [
        'AppleGothic',           # macOS 기본
        'Apple SD Gothic Neo',   # macOS
        'NanumGothic',          # 나눔고딕
        'NanumBarunGothic',     # 나눔바른고딕
        'Malgun Gothic',        # 맑은 고딕 (Windows)
        'Noto Sans CJK KR',     # 구글 Noto
        'DejaVu Sans'           # 폴백 (한글 안되지만 깨지지는 않음)
    ]
    
    # 시스템에 설치된 폰트 목록
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    
    # 사용 가능한 첫 번째 폰트 찾기
    selected_font = None
    for font in font_candidates:
        if font in available_fonts:
            selected_font = font
            break
    
    # 폰트 설정
    if selected_font:
        plt.rc('font', family=selected_font)
        print(f"✅ 한글 폰트 설정: {selected_font}")
    else:
        # 폰트를 찾지 못한 경우 - 유니코드 사용
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 한글 폰트를 찾지 못했습니다. 기본 설정을 사용합니다.")
        print("   한글이 깨질 수 있습니다. 나눔고딕 설치를 권장합니다.")
    
    # 마이너스 기호 깨짐 방지
    plt.rcParams['axes.unicode_minus'] = False
    
    return selected_font

# 한글 폰트 설정
selected_font = set_korean_font()

# 스타일 설정
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)

print("="*80)
print("📊 서울시 5대 범죄 위험 시간대 가설 검증")
print("="*80)

# ============================================
# 데이터 로드
# ============================================

df_time = pd.read_csv('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/ouput_2crime_time_distribution.csv', encoding='cp949')
df_place = pd.read_csv('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/ouput_2crime_place_distribution.csv', encoding='cp949')

print("\n✅ 데이터 로드 완료")
print(f"   - 시간 분포: {df_time.shape}")
print(f"   - 장소 분포: {df_place.shape}")


# ============================================
# 1. 장소별 범죄 유형 분포 분석
# ============================================

print("\n" + "="*80)
print("[분석 1] 장소별 범죄 유형 분포")
print("="*80)

# 데이터 준비
place_cols = ['주거지구', '업무지구', '다중이용시설', '기타']
df_place_analysis = df_place.set_index('범죄분류')[place_cols]

# 비율 계산
df_place_pct = df_place_analysis.div(df_place_analysis.sum(axis=0), axis=1) * 100

print("\n📊 장소별 범죄 비율 (%):")
print(df_place_pct.round(2))

# 1-1. Heatmap
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Heatmap - 절대값
sns.heatmap(df_place_analysis, annot=True, fmt=',', cmap='YlOrRd', 
            cbar_kws={'label': 'Count'}, ax=axes[0], linewidths=0.5)
axes[0].set_title('Place-wise Crime Count (Absolute)', fontsize=16, weight='bold', pad=20)
axes[0].set_xlabel('Location Category', fontsize=12, weight='bold')
axes[0].set_ylabel('Crime Type', fontsize=12, weight='bold')

# Heatmap - 비율
sns.heatmap(df_place_pct, annot=True, fmt='.1f', cmap='RdYlGn_r', 
            cbar_kws={'label': 'Percentage (%)'}, ax=axes[1], linewidths=0.5)
axes[1].set_title('Place-wise Crime Ratio (%)', fontsize=16, weight='bold', pad=20)
axes[1].set_xlabel('Location Category', fontsize=12, weight='bold')
axes[1].set_ylabel('Crime Type', fontsize=12, weight='bold')

plt.tight_layout()
plt.savefig('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/01_place_crime_heatmap_fixed.png', dpi=300, bbox_inches='tight')
print("\n✅ 그래프 저장: 01_place_crime_heatmap_fixed.png")
plt.close()

# 1-2. Stacked Bar Chart
fig, ax = plt.subplots(figsize=(14, 8))

# 영문 라벨 사용 (한글 깨짐 방지)
df_place_pct_eng = df_place_pct.copy()
df_place_pct_eng.index = ['Theft', 'Murder', 'Robbery', 'Sex Crime', 'Violence']

df_place_pct_eng.T.plot(kind='bar', stacked=True, ax=ax, 
                         color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8'])

ax.set_title('Crime Type Distribution by Location', fontsize=18, weight='bold', pad=20)
ax.set_xlabel('Location Category', fontsize=14, weight='bold')
ax.set_ylabel('Percentage (%)', fontsize=14, weight='bold')
ax.legend(title='Crime Type', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=11)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center')
ax.grid(axis='y', alpha=0.3)

# 각 바에 비율 표시
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f%%', label_type='center', fontsize=9, weight='bold')

plt.tight_layout()
plt.savefig('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/02_place_crime_stacked_fixed.png', dpi=300, bbox_inches='tight')
print("✅ 그래프 저장: 02_place_crime_stacked_fixed.png")
plt.close()

# 주요 인사이트 추출
print("\n📌 [인사이트 1] 장소별 주요 범죄")
print("-"*80)
for place in place_cols:
    top_crime = df_place_pct[place].idxmax()
    top_pct = df_place_pct[place].max()
    print(f"   {place:12s}: {top_crime} (비율: {top_pct:.1f}%)")


# ============================================
# 2. 범죄 유형별 시간대별 발생 패턴
# ============================================

print("\n" + "="*80)
print("[분석 2] 범죄 유형별 시간대별 발생 패턴")
print("="*80)

# 시간대 컬럼 정리
time_cols = [col for col in df_time.columns if col != '범죄분류' and col != '미상']

# 데이터 준비
df_time_analysis = df_time.set_index('범죄분류')[time_cols]

# 시간대 라벨
time_labels = ['00-02', '03-05', '06-08', '09-11', '12-14', '15-17', '18-20', '21-23']

print("\n📊 범죄별 시간대별 발생 건수:")
print(df_time_analysis)

# 영문 범죄명 매핑
crime_name_map = {
    '절도': 'Theft',
    '살인': 'Murder',
    '강도': 'Robbery',
    '성폭력': 'Sex Crime',
    '폭력': 'Violence'
}

# 2-1. Line Chart - 절대값
fig, ax = plt.subplots(figsize=(16, 8))

for crime in df_time_analysis.index:
    crime_eng = crime_name_map.get(crime, crime)
    ax.plot(time_labels, df_time_analysis.loc[crime], 
            marker='o', linewidth=2.5, markersize=8, label=crime_eng)

ax.set_title('Crime Pattern by Time (Absolute Count)', fontsize=18, weight='bold', pad=20)
ax.set_xlabel('Time Period (Hour)', fontsize=14, weight='bold')
ax.set_ylabel('Crime Count', fontsize=14, weight='bold')
ax.legend(title='Crime Type', fontsize=12, title_fontsize=13, loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_ylim(bottom=0)

# 피크 타임 표시
for crime, crime_eng in [('절도', 'Theft'), ('폭력', 'Violence')]:
    peak_idx = df_time_analysis.loc[crime].argmax()
    peak_value = df_time_analysis.loc[crime].max()
    ax.annotate(f'{crime_eng} Peak\n{peak_value:,}', 
                xy=(peak_idx, peak_value), 
                xytext=(peak_idx, peak_value + 5000),
                ha='center', fontsize=10, weight='bold',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', lw=2, color='red'))

plt.tight_layout()
plt.savefig('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/03_time_pattern_absolute_fixed.png', dpi=300, bbox_inches='tight')
print("\n✅ 그래프 저장: 03_time_pattern_absolute_fixed.png")
plt.close()

# 2-2. Line Chart - 정규화
df_time_pct = df_time_analysis.div(df_time_analysis.sum(axis=1), axis=0) * 100

fig, ax = plt.subplots(figsize=(16, 8))

for crime in df_time_pct.index:
    crime_eng = crime_name_map.get(crime, crime)
    ax.plot(time_labels, df_time_pct.loc[crime], 
            marker='o', linewidth=2.5, markersize=8, label=crime_eng)

ax.set_title('Crime Pattern by Time (Percentage)', fontsize=18, weight='bold', pad=20)
ax.set_xlabel('Time Period (Hour)', fontsize=14, weight='bold')
ax.set_ylabel('Percentage (%)', fontsize=14, weight='bold')
ax.legend(title='Crime Type', fontsize=12, title_fontsize=13, loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.savefig('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/04_time_pattern_percentage_fixed.png', dpi=300, bbox_inches='tight')
print("✅ 그래프 저장: 04_time_pattern_percentage_fixed.png")
plt.close()

# 피크 타임 분석
print("\n📌 [인사이트 2] 범죄별 피크 타임")
print("-"*80)
for crime in df_time_analysis.index:
    peak_idx = df_time_analysis.loc[crime].argmax()
    peak_time = time_labels[peak_idx]
    peak_count = df_time_analysis.loc[crime].max()
    
    min_idx = df_time_analysis.loc[crime].argmin()
    min_time = time_labels[min_idx]
    min_count = df_time_analysis.loc[crime].min()
    
    print(f"   {crime:8s}: Peak {peak_time} ({peak_count:>6,}) / Low {min_time} ({min_count:>6,})")


# ============================================
# 3. 가설 검증 통합 분석
# ============================================

print("\n\n" + "="*80)
print("[분석 3] 가설 검증 - 통합 분석")
print("="*80)

# 3-1. 장소별 주요 범죄 식별
print("\n🔍 STEP 1: 각 장소에서 가장 많이 발생하는 범죄 식별")
print("-"*80)

place_main_crimes = {}
for place in ['주거지구', '업무지구', '다중이용시설']:
    place_data = df_place.set_index('범죄분류')[place]
    top_crimes = place_data.nlargest(2)
    place_main_crimes[place] = top_crimes
    
    print(f"\n   {place}:")
    for crime, count in top_crimes.items():
        pct = (count / place_data.sum()) * 100
        print(f"      {crime}: {count:>6,} ({pct:>5.1f}%)")

# 3-2. 피크 타임 분석
print("\n\n🔍 STEP 2: 주요 범죄의 피크 타임 분석")
print("-"*80)

crime_peak_times = {}
for crime in df_time['범죄분류']:
    crime_data = df_time_analysis.loc[crime]
    peak_idx = crime_data.argmax()
    peak_time = time_labels[peak_idx]
    peak_count = crime_data.max()
    crime_peak_times[crime] = (peak_time, peak_count)
    
    print(f"   {crime:8s}: {peak_time} ({peak_count:,})")

# 3-3. 통합 시각화 (영문 버전)
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)


# 서브플롯 1: 장소별 범죄 구성
ax1 = fig.add_subplot(gs[0, :])
df_place_pct_eng[['주거지구', '업무지구', '다중이용시설']].T.plot(
    kind='bar', stacked=True, ax=ax1,
    color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8'])
ax1.set_title('Crime Composition by Location', fontsize=16, weight='bold')
ax1.set_xlabel('Location Category', fontsize=12, weight='bold')
ax1.set_ylabel('Percentage (%)', fontsize=12, weight='bold')
ax1.set_xticklabels(['Residential', 'Business', 'Public Facility'], rotation=0)
ax1.legend(title='Crime Type', bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(axis='y', alpha=0.3)

# 서브플롯 2-4: 장소별 시간 패턴
locations = [
    ('주거지구', 'Residential', gs[1, 0]),
    ('업무지구', 'Business', gs[1, 1]),
    ('다중이용시설', 'Public Facility', gs[2, 0])
]

for place_kr, place_en, position in locations:
    ax = fig.add_subplot(position)
    for crime in place_main_crimes[place_kr].index:
        crime_eng = crime_name_map.get(crime, crime)
        ax.plot(time_labels, df_time_analysis.loc[crime], 
                marker='o', linewidth=2.5, markersize=8, label=crime_eng)
    ax.set_title(f'{place_en} Crime Time Pattern', fontsize=14, weight='bold')
    ax.set_xlabel('Time Period', fontsize=11)
    ax.set_ylabel('Crime Count', fontsize=11)
    ax.legend()
    ax.grid(True, alpha=0.3)

# 서브플롯 5: 가설 검증 요약
ax5 = fig.add_subplot(gs[2, 1])
ax5.axis('off')

residential_crime = place_main_crimes['주거지구'].idxmax()
business_crime = place_main_crimes['업무지구'].idxmax()
public_crime = place_main_crimes['다중이용시설'].idxmax()

residential_peak = crime_peak_times[residential_crime][0]
business_peak = crime_peak_times[business_crime][0]
public_peak = crime_peak_times[public_crime][0]

summary_text = f"""
Hypothesis Verification Result

✅ Crime Type Difference by Location
   • Residential: {residential_crime}
   • Business: {business_crime}
   • Public: {public_crime}

✅ Peak Time Difference
   • Residential: {residential_peak}
   • Business: {business_peak}
   • Public: {public_peak}

🎯 Conclusion: Hypothesis Partially Verified
   Different peak times for different crimes
"""

ax5.text(0.1, 0.5, summary_text, fontsize=12, verticalalignment='center',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
         family='monospace')

plt.savefig('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/05_hypothesis_verification_fixed.png', dpi=300, bbox_inches='tight')
print("\n✅ 통합 그래프 저장: 05_hypothesis_verification_fixed.png")
plt.close()


# ============================================
# 4. 최종 인사이트 레포트
# ============================================

print("\n\n" + "="*80)
print("📋 최종 인사이트 레포트")
print("="*80)

residential_count = place_main_crimes['주거지구'].max()
business_count = place_main_crimes['업무지구'].max()
public_count = place_main_crimes['다중이용시설'].max()

report = f"""

🎯 가설 검증 결과: ✅ 부분 성립

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1️⃣ 장소별 주요 범죄 유형 분석

   📍 주거지구 (Residential)
      • 최다 범죄: {residential_crime} ({residential_count:,}건)
      • 특징: 사생활 공간에서 폭력 범죄 집중
   
   📍 업무지구 (Business) - 가장 안전
      • 최다 범죄: {business_crime} ({business_count:,}건)
      • 특징: 전체 범죄 발생률 가장 낮음 ({df_place['업무지구'].sum():,}건)
   
   📍 다중이용시설 (Public Facility) - 가장 위험
      • 최다 범죄: {public_crime} ({public_count:,}건)
      • 특징: 전체 범죄의 62.5% 집중 ({df_place['다중이용시설'].sum():,}건)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2️⃣ 위험 골든타임 (Peak Hours) 분석

   ⏰ 주거지구: {residential_peak} ({residential_crime})
   ⏰ 업무지구: {business_peak} ({business_crime})
   ⏰ 다중이용시설: {public_peak} ({public_crime})

   💡 핵심 발견:
      • 절도 피크: 15-17시 (오후 시간대)
      • 폭력 피크: 21-23시 (야간 시간대)
      → 범죄 유형별 시간대 패턴 명확히 다름!

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

3️⃣ 주요 발견 사항

   💡 인사이트 1: 모든 장소에서 폭력이 1위
      → 장소별 범죄 유형 차이는 제한적
      
   💡 인사이트 2: 범죄별 시간대 패턴 명확
      → 절도(15-17시) vs 폭력(21-23시)
      
   💡 인사이트 3: 업무지구가 가장 안전
      → 총 {df_place['업무지구'].sum():,}건 (전체의 3.6%)
      → 주거지구 대비 약 5.5배 안전

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

4️⃣ 정책 제언

   🔒 시간대별 맞춤 치안 전략
      • 15-17시: 상업지구 절도 집중 순찰
      • 21-23시: 주거/유흥가 폭력 예방 강화
   
   📍 장소별 안전 대책
      • 업무지구: 현 수준 유지
      • 주거지구: 야간 가정폭력 예방
      • 다중이용시설: CCTV/조명 확충

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✅ 결론: 범죄 유형별 시간대 패턴이 다르므로
        시간대별 맞춤형 치안 전략 수립 가능

"""

print(report)

# 레포트 저장
with open('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/insight_report_fixed.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n✅ 인사이트 레포트 저장: insight_report_fixed.txt")

print("\n" + "="*80)
print("✅ 가설 검증 및 시각화 완료! (한글 폰트 문제 해결)")
print("="*80)

print("\n📊 생성된 파일:")
print("   1. 01_place_crime_heatmap_fixed.png")
print("   2. 02_place_crime_stacked_fixed.png")
print("   3. 03_time_pattern_absolute_fixed.png")
print("   4. 04_time_pattern_percentage_fixed.png")
print("   5. 05_hypothesis_verification_fixed.png")
print("   6. insight_report_fixed.txt")
print("\n💡 그래프의 축 레이블과 범례는 영문으로 표시됩니다.")

✅ 한글 폰트 설정: AppleGothic
📊 서울시 5대 범죄 위험 시간대 가설 검증

✅ 데이터 로드 완료
   - 시간 분포: (5, 10)
   - 장소 분포: (5, 5)

[분석 1] 장소별 범죄 유형 분포

📊 장소별 범죄 비율 (%):
       주거지구   업무지구  다중이용시설     기타
범죄분류                             
절도    31.10  40.14   45.78  56.92
살인     0.48   0.13    0.10   0.25
강도     0.24   0.14    0.19   0.20
성폭력    8.34   8.70    6.82   9.44
폭력    59.84  50.88   47.10  33.18

✅ 그래프 저장: 01_place_crime_heatmap_fixed.png
✅ 그래프 저장: 02_place_crime_stacked_fixed.png

📌 [인사이트 1] 장소별 주요 범죄
--------------------------------------------------------------------------------
   주거지구        : 폭력 (비율: 59.8%)
   업무지구        : 폭력 (비율: 50.9%)
   다중이용시설      : 폭력 (비율: 47.1%)
   기타          : 절도 (비율: 56.9%)

[분석 2] 범죄 유형별 시간대별 발생 패턴

📊 범죄별 시간대별 발생 건수:
      00시00분-02시59분  03시00분-05시59분  06시00분-08시59분  09시00분-11시59분  \
범죄분류                                                               
절도            13095          14176          13104          21763   
살인               49             73             49      

In [4]:
"""
서울시 5대 범죄 위험 시간대 가설 검증 및 시각화
(한글 깨짐 완전 해결 버전 - 모든 텍스트를 영문으로 변환)
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ============================================
# 설정: 모든 텍스트를 영문으로 처리
# ============================================

# 한글-영문 매핑
CRIME_MAP = {
    '절도': 'Theft',
    '살인': 'Murder',
    '강도': 'Robbery',
    '성폭력': 'Sex Crime',
    '폭력': 'Violence'
}

PLACE_MAP = {
    '주거지구': 'Residential',
    '업무지구': 'Business',
    '다중이용시설': 'Public Facility',
    '기타': 'Others'
}

TIME_MAP = {
    '00시00분-02시59분': '00-02h',
    '03시00분-05시59분': '03-05h',
    '06시00분-08시59분': '06-08h',
    '09시00분-11시59분': '09-11h',
    '12시00분-14시59분': '12-14h',
    '15시00분-17시59분': '15-17h',
    '18시00분-20시59분': '18-20h',
    '21시00분-23시59분': '21-23h'
}

# 스타일 설정
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 11

print("="*80)
print("📊 Seoul Crime Risk Analysis - Hypothesis Verification")
print("="*80)

# ============================================
# 데이터 로드
# ============================================

# 파일 경로 수정 필요
df_time = pd.read_csv('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/ouput_2crime_time_distribution.csv', encoding='cp949')
df_place = pd.read_csv('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/ouput_2crime_place_distribution.csv', encoding='cp949')

print("\n✅ Data Loaded Successfully")
print(f"   - Time Distribution: {df_time.shape}")
print(f"   - Place Distribution: {df_place.shape}")

# ============================================
# 데이터 전처리: 한글 → 영문 변환
# ============================================

# 범죄 유형명 변환
df_time['Crime Type'] = df_time['범죄분류'].map(CRIME_MAP)
df_place['Crime Type'] = df_place['범죄분류'].map(CRIME_MAP)

# 장소 컬럼명 변환
place_cols_kr = ['주거지구', '업무지구', '다중이용시설', '기타']
place_cols_en = [PLACE_MAP[col] for col in place_cols_kr]

# 시간대 컬럼명 변환
time_cols_kr = [col for col in df_time.columns if col not in ['범죄분류', '미상', 'Crime Type']]
time_labels = ['00-02', '03-05', '06-08', '09-11', '12-14', '15-17', '18-20', '21-23']

print("\n✅ Data Preprocessing Complete (Korean → English)")

# ============================================
# 1. 장소별 범죄 유형 분포 분석
# ============================================

print("\n" + "="*80)
print("[Analysis 1] Crime Distribution by Location")
print("="*80)

# 영문 데이터프레임 생성
df_place_en = df_place.set_index('Crime Type')[place_cols_kr].copy()
df_place_en.columns = place_cols_en

# 비율 계산
df_place_pct = df_place_en.div(df_place_en.sum(axis=0), axis=1) * 100

print("\n📊 Crime Ratio by Location (%):")
print(df_place_pct.round(2))

# 1-1. Heatmap
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Heatmap - Absolute Count
sns.heatmap(df_place_en, annot=True, fmt=',', cmap='YlOrRd', 
            cbar_kws={'label': 'Crime Count'}, ax=axes[0], 
            linewidths=0.5, linecolor='white')
axes[0].set_title('Crime Count by Location (Absolute)', 
                   fontsize=16, weight='bold', pad=20)
axes[0].set_xlabel('Location Category', fontsize=12, weight='bold')
axes[0].set_ylabel('Crime Type', fontsize=12, weight='bold')
axes[0].tick_params(axis='x', rotation=0)
axes[0].tick_params(axis='y', rotation=0)

# Heatmap - Percentage
sns.heatmap(df_place_pct, annot=True, fmt='.1f', cmap='RdYlGn_r', 
            cbar_kws={'label': 'Percentage (%)'}, ax=axes[1], 
            linewidths=0.5, linecolor='white')
axes[1].set_title('Crime Ratio by Location (%)', 
                   fontsize=16, weight='bold', pad=20)
axes[1].set_xlabel('Location Category', fontsize=12, weight='bold')
axes[1].set_ylabel('Crime Type', fontsize=12, weight='bold')
axes[1].tick_params(axis='x', rotation=0)
axes[1].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.savefig('01_place_crime_heatmap_english.png', dpi=300, bbox_inches='tight')
print("\n✅ Graph Saved: 01_place_crime_heatmap_english.png")
plt.close()

# 1-2. Stacked Bar Chart
fig, ax = plt.subplots(figsize=(14, 8))

df_place_pct.T.plot(kind='bar', stacked=True, ax=ax, 
                    color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8'],
                    edgecolor='white', linewidth=1.5)

ax.set_title('Crime Type Distribution by Location', 
             fontsize=18, weight='bold', pad=20)
ax.set_xlabel('Location Category', fontsize=14, weight='bold')
ax.set_ylabel('Percentage (%)', fontsize=14, weight='bold')
ax.legend(title='Crime Type', bbox_to_anchor=(1.05, 1), 
          loc='upper left', fontsize=11, title_fontsize=12)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center')
ax.grid(axis='y', alpha=0.3)
ax.set_ylim(0, 100)

# 바에 비율 표시
for container in ax.containers:
    labels = [f'{v:.1f}%' if v > 5 else '' for v in container.datavalues]
    ax.bar_label(container, labels=labels, label_type='center', 
                fontsize=9, weight='bold', color='white')

plt.tight_layout()
plt.savefig('02_place_crime_stacked_english.png', dpi=300, bbox_inches='tight')
print("✅ Graph Saved: 02_place_crime_stacked_english.png")
plt.close()

# 주요 인사이트
print("\n📌 [Insight 1] Main Crime by Location")
print("-"*80)
for place in place_cols_en:
    top_crime = df_place_pct[place].idxmax()
    top_pct = df_place_pct[place].max()
    count = df_place_en[place].max()
    print(f"   {place:18s}: {top_crime:12s} ({top_pct:5.1f}% | {count:>6,} cases)")

# ============================================
# 2. 범죄 유형별 시간대별 발생 패턴
# ============================================

print("\n" + "="*80)
print("[Analysis 2] Crime Pattern by Time")
print("="*80)

# 영문 데이터프레임 생성
df_time_en = df_time.set_index('Crime Type')[time_cols_kr].copy()
df_time_en.columns = time_labels

print("\n📊 Crime Count by Time Period:")
print(df_time_en)

# 2-1. Line Chart - Absolute Count
fig, ax = plt.subplots(figsize=(16, 8))

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']
for i, crime in enumerate(df_time_en.index):
    ax.plot(time_labels, df_time_en.loc[crime], 
            marker='o', linewidth=3, markersize=10, 
            label=crime, color=colors[i])

ax.set_title('Crime Pattern by Time (Absolute Count)', 
             fontsize=18, weight='bold', pad=20)
ax.set_xlabel('Time Period (Hour)', fontsize=14, weight='bold')
ax.set_ylabel('Crime Count', fontsize=14, weight='bold')
ax.legend(title='Crime Type', fontsize=12, title_fontsize=13, 
          loc='upper left', frameon=True, shadow=True)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_ylim(bottom=0)

# 피크 타임 강조
for crime, color in [('Theft', '#45B7D1'), ('Violence', '#FF6B6B')]:
    if crime in df_time_en.index:
        peak_idx = df_time_en.loc[crime].argmax()
        peak_value = df_time_en.loc[crime].max()
        ax.annotate(f'{crime} Peak\n{peak_value:,}', 
                    xy=(peak_idx, peak_value), 
                    xytext=(peak_idx, peak_value + 5000),
                    ha='center', fontsize=11, weight='bold',
                    bbox=dict(boxstyle='round,pad=0.5', 
                             facecolor='yellow', alpha=0.8, edgecolor=color, linewidth=2),
                    arrowprops=dict(arrowstyle='->', lw=2.5, color=color))

plt.tight_layout()
plt.savefig('03_time_pattern_absolute_english.png', dpi=300, bbox_inches='tight')
print("\n✅ Graph Saved: 03_time_pattern_absolute_english.png")
plt.close()

# 2-2. Line Chart - Normalized Percentage
df_time_pct = df_time_en.div(df_time_en.sum(axis=1), axis=0) * 100

fig, ax = plt.subplots(figsize=(16, 8))

for i, crime in enumerate(df_time_pct.index):
    ax.plot(time_labels, df_time_pct.loc[crime], 
            marker='o', linewidth=3, markersize=10, 
            label=crime, color=colors[i])

ax.set_title('Crime Pattern by Time (Percentage)', 
             fontsize=18, weight='bold', pad=20)
ax.set_xlabel('Time Period (Hour)', fontsize=14, weight='bold')
ax.set_ylabel('Percentage (%)', fontsize=14, weight='bold')
ax.legend(title='Crime Type', fontsize=12, title_fontsize=13, 
          loc='upper left', frameon=True, shadow=True)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.savefig('04_time_pattern_percentage_english.png', dpi=300, bbox_inches='tight')
print("✅ Graph Saved: 04_time_pattern_percentage_english.png")
plt.close()

# 피크 타임 분석
print("\n📌 [Insight 2] Peak Time by Crime Type")
print("-"*80)
for crime in df_time_en.index:
    peak_idx = df_time_en.loc[crime].argmax()
    peak_time = time_labels[peak_idx]
    peak_count = df_time_en.loc[crime].max()
    
    min_idx = df_time_en.loc[crime].argmin()
    min_time = time_labels[min_idx]
    min_count = df_time_en.loc[crime].min()
    
    print(f"   {crime:12s}: Peak at {peak_time} ({peak_count:>6,}) "
          f"/ Low at {min_time} ({min_count:>6,})")

# ============================================
# 3. 통합 분석 및 시각화
# ============================================

print("\n\n" + "="*80)
print("[Analysis 3] Integrated Analysis - Hypothesis Verification")
print("="*80)

# 3-1. 장소별 주요 범죄
print("\n🔍 STEP 1: Identify Main Crimes by Location")
print("-"*80)

place_main_crimes = {}
for place in ['Residential', 'Business', 'Public Facility']:
    place_data = df_place_en[place]
    top_crimes = place_data.nlargest(2)
    place_main_crimes[place] = top_crimes
    
    print(f"\n   {place}:")
    for crime, count in top_crimes.items():
        pct = (count / place_data.sum()) * 100
        print(f"      {crime:12s}: {count:>6,} ({pct:>5.1f}%)")

# 3-2. 피크 타임 분석
print("\n\n🔍 STEP 2: Peak Time Analysis by Crime Type")
print("-"*80)

crime_peak_times = {}
for crime in df_time_en.index:
    crime_data = df_time_en.loc[crime]
    peak_idx = crime_data.argmax()
    peak_time = time_labels[peak_idx]
    peak_count = crime_data.max()
    crime_peak_times[crime] = (peak_time, peak_count)
    
    print(f"   {crime:12s}: {peak_time} ({peak_count:,} cases)")

# 3-3. 통합 시각화
fig = plt.figure(figsize=(20, 14))
gs = fig.add_gridspec(3, 2, hspace=0.35, wspace=0.3)

# 서브플롯 1: 장소별 범죄 구성
ax1 = fig.add_subplot(gs[0, :])
df_place_pct[['Residential', 'Business', 'Public Facility']].T.plot(
    kind='bar', stacked=True, ax=ax1,
    color=colors, edgecolor='white', linewidth=1.5)
ax1.set_title('Crime Composition by Location', fontsize=18, weight='bold', pad=20)
ax1.set_xlabel('Location Category', fontsize=14, weight='bold')
ax1.set_ylabel('Percentage (%)', fontsize=14, weight='bold')
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
ax1.legend(title='Crime Type', bbox_to_anchor=(1.05, 1), loc='upper left',
          fontsize=11, title_fontsize=12)
ax1.grid(axis='y', alpha=0.3)
ax1.set_ylim(0, 100)

# 서브플롯 2-4: 장소별 시간 패턴
locations = [
    ('Residential', gs[1, 0]),
    ('Business', gs[1, 1]),
    ('Public Facility', gs[2, 0])
]

for place, position in locations:
    ax = fig.add_subplot(position)
    for i, crime in enumerate(place_main_crimes[place].index):
        ax.plot(time_labels, df_time_en.loc[crime], 
                marker='o', linewidth=2.5, markersize=8, 
                label=crime, color=colors[i])
    ax.set_title(f'{place} Crime Time Pattern', fontsize=14, weight='bold')
    ax.set_xlabel('Time Period', fontsize=11, weight='bold')
    ax.set_ylabel('Crime Count', fontsize=11, weight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

# 서브플롯 5: 가설 검증 요약
ax5 = fig.add_subplot(gs[2, 1])
ax5.axis('off')

residential_crime = place_main_crimes['Residential'].idxmax()
business_crime = place_main_crimes['Business'].idxmax()
public_crime = place_main_crimes['Public Facility'].idxmax()

residential_peak = crime_peak_times[residential_crime][0]
business_peak = crime_peak_times[business_crime][0]
public_peak = crime_peak_times[public_crime][0]

summary_text = f"""
╔══════════════════════════════════════════╗
║  HYPOTHESIS VERIFICATION RESULT          ║
╚══════════════════════════════════════════╝

✅ Crime Type Difference by Location
   • Residential:      {residential_crime}
   • Business:         {business_crime}
   • Public Facility:  {public_crime}

✅ Peak Time Difference
   • Residential:      {residential_peak}
   • Business:         {business_peak}
   • Public Facility:  {public_peak}

🎯 CONCLUSION
   Hypothesis Partially Verified!
   
   → Different peak times exist
     for different crime types
   
   → Location affects crime types
     but patterns are similar
"""

ax5.text(0.05, 0.5, summary_text, fontsize=11, verticalalignment='center',
         bbox=dict(boxstyle='round,pad=1', facecolor='wheat', alpha=0.7,
                  edgecolor='black', linewidth=2),
         family='monospace', weight='bold')

plt.savefig('05_hypothesis_verification_english.png', dpi=300, bbox_inches='tight')
print("\n✅ Integrated Graph Saved: 05_hypothesis_verification_english.png")
plt.close()

# ============================================
# 4. 최종 리포트
# ============================================

print("\n\n" + "="*80)
print("📋 FINAL INSIGHT REPORT")
print("="*80)

residential_count = place_main_crimes['Residential'].max()
business_count = place_main_crimes['Business'].max()
public_count = place_main_crimes['Public Facility'].max()

total_residential = df_place_en['Residential'].sum()
total_business = df_place_en['Business'].sum()
total_public = df_place_en['Public Facility'].sum()

report = f"""

🎯 HYPOTHESIS VERIFICATION RESULT: ✅ PARTIALLY VERIFIED

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1️⃣ MAIN CRIME TYPE BY LOCATION

   📍 Residential Area
      • Top Crime: {residential_crime} ({residential_count:,} cases)
      • Characteristics: Violence concentrated in private spaces
      • Total Cases: {total_residential:,}
   
   📍 Business District - SAFEST
      • Top Crime: {business_crime} ({business_count:,} cases)
      • Characteristics: Lowest crime rate overall
      • Total Cases: {total_business:,} (Only 3.6% of total)
   
   📍 Public Facility - MOST DANGEROUS
      • Top Crime: {public_crime} ({public_count:,} cases)
      • Characteristics: 62.5% of all crimes concentrated here
      • Total Cases: {total_public:,}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2️⃣ PEAK HOURS (GOLDEN TIME) ANALYSIS

   ⏰ Residential:      {residential_peak} ({residential_crime})
   ⏰ Business:         {business_peak} ({business_crime})
   ⏰ Public Facility:  {public_peak} ({public_crime})

   💡 KEY FINDINGS:
      • Theft Peak: 15-17h (afternoon)
      • Violence Peak: 21-23h (night)
      → Clear time pattern difference by crime type!

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

3️⃣ KEY DISCOVERIES

   💡 Insight 1: Violence is #1 in all locations
      → Limited crime type difference by location
      
   💡 Insight 2: Clear time patterns by crime type
      → Theft (15-17h) vs Violence (21-23h)
      
   💡 Insight 3: Business districts are safest
      → Total {total_business:,} cases (3.6% of all)
      → 5.5x safer than residential areas

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

4️⃣ POLICY RECOMMENDATIONS

   🔒 Time-based Security Strategy
      • 15-17h: Intensive theft patrol in commercial areas
      • 21-23h: Violence prevention in residential/nightlife areas
   
   📍 Location-based Safety Measures
      • Business: Maintain current level
      • Residential: Night domestic violence prevention
      • Public Facility: Install more CCTV/lighting

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✅ CONCLUSION
   Different time patterns exist for different crime types
   → Customized time-based security strategies are feasible

"""

print(report)

# 리포트 저장
with open('insight_report_english.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n✅ Insight Report Saved: insight_report_english.txt")

print("\n" + "="*80)
print("✅ ANALYSIS COMPLETE!")
print("="*80)

print("\n📊 Generated Files:")
print("   1. 01_place_crime_heatmap_english.png")
print("   2. 02_place_crime_stacked_english.png")
print("   3. 03_time_pattern_absolute_english.png")
print("   4. 04_time_pattern_percentage_english.png")
print("   5. 05_hypothesis_verification_english.png")
print("   6. insight_report_english.txt")
print("\n💡 All labels and legends are in English (No Korean font issues!)")
print("="*80)

📊 Seoul Crime Risk Analysis - Hypothesis Verification

✅ Data Loaded Successfully
   - Time Distribution: (5, 10)
   - Place Distribution: (5, 5)

✅ Data Preprocessing Complete (Korean → English)

[Analysis 1] Crime Distribution by Location

📊 Crime Ratio by Location (%):
            Residential  Business  Public Facility  Others
Crime Type                                                
Theft             31.10     40.14            45.78   56.92
Murder             0.48      0.13             0.10    0.25
Robbery            0.24      0.14             0.19    0.20
Sex Crime          8.34      8.70             6.82    9.44
Violence          59.84     50.88            47.10   33.18

✅ Graph Saved: 01_place_crime_heatmap_english.png
✅ Graph Saved: 02_place_crime_stacked_english.png

📌 [Insight 1] Main Crime by Location
--------------------------------------------------------------------------------
   Residential       : Violence     ( 59.8% | 49,634 cases)
   Business          : Violence   